In [1]:
from qiskit.providers.fake_provider import FakeBackend
from qiskit.providers.models import BackendProperties, QasmBackendConfiguration
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit.quantum_info import hellinger_fidelity
from qiskit import transpile, QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService
# from qiskit_algorithms import s

from bqskit.passes import QSearchSynthesisPass, QuickPartitioner, ForEachBlockPass, ExtractMeasurements, RestoreMeasurements, UnfoldPass
from bqskit.compiler import Compiler

import json
import re
from datetime import datetime

provider = AerSimulator()



/var/folders/7x/9zsm44vd7h7dhh4b6cdzjjvc0000gn/T/ipykernel_13940/3375778465.py:2: DeprecationWarning: qiskit.providers.models is deprecated since Qiskit 1.2 and will be removed in Qiskit 2.0. With the removal of Qobj, there is no need for these schema-conformant objects. If you still need to use them, it could be because you are using a BackendV1, which is also deprecated in favor of BackendV2.
  from qiskit.providers.models import BackendProperties, QasmBackendConfiguration


In [33]:
# If you want to import backend from .json
def convert_datetime_to_iso(match):
    year, month, day, hour, minute, second = map(int, match.groups())
    datetime_obj = datetime(year, month, day, hour, minute, second)
    return '"{}"'.format(datetime_obj.isoformat())



def get_json_backend(backend_filename: str):
    # Using a backend json, construct a fake backend
    
    with open(backend_filename, "r") as file:
        # Read the data as a string.

        backend_data = json.load(file)
        # Convert the data to a Python dictionary.
        # Read the data as a dictionary (key:value pairs).
        datetime_pattern = r'"datetime\((\d+), (\d+), (\d+), (\d+), (\d+), (\d+)\)"'
        # Replace datetime strings with ISO format strings.
        backend_data = re.sub(datetime_pattern, convert_datetime_to_iso, backend_data)

        backend_data = json.loads(backend_data)

        backend_data = backend_data["devise_properties"]

    props = BackendProperties.from_dict(data=backend_data)
    
    
    basis_gates = set()
    for gate in props.gates:
        basis_gates.add(gate.gate)
    basis_gates = list(basis_gates)

    # Determine the two-qubit entangling gate used to generate the coupling map.

    two_qubit_gate = None
    if "cx" in basis_gates:
        two_qubit_gate = "cx"
    elif "ecr" in basis_gates:
        two_qubit_gate = "ecr"
    elif "cz" in basis_gates:
        two_qubit_gate = "cz"
    else:
        raise ValueError("I do not know what 2Q gate the backend is using")

    # Iterate over the two-qubit gates to obtain the coupling map.

    coupling_map = []
    for gate in props.gates:
        if gate.gate == two_qubit_gate:
            coupling_map.append(gate.qubits)
    # Form a configuration object from the data in the properties.

    config = QasmBackendConfiguration(
        backend_name=props.backend_name,
        backend_version=props.backend_version,
        n_qubits=len(props.qubits),
        basis_gates=basis_gates,
        gates=basis_gates,  # This is not really valid, but not using it.
        local=True,
        simulator=False,
        conditional=True,
        open_pulse=False,
        memory=True,
        max_shots=None,
        coupling_map=coupling_map,
    )

    # Overload the properties method on the FakeBackend to return our properties.

    def properties():
        return props

    # Form the actual FakeBackend to return.

    fake_backend = FakeBackend(config)
    fake_backend.properties = properties

    return fake_backend

In [34]:
# Simulate the circuit using a specified simulator

def simulate_circuit(circuit, simulator):

    result = simulator.run(circuit, shots=8192).result()
    counts = result.get_counts()

    return counts

In [2]:
# Define the service and backend
service = QiskitRuntimeService()
backend = service.get_backend('ibm_nazca')

# Define the simulators
noise_model = NoiseModel.from_backend(backend)
noise_simulator = AerSimulator(noise_model = noise_model)
reg_sim = AerSimulator()

# Define original machine aware circuit
circ = QuantumCircuit.from_qasm_file('original_shors.qasm')
circ_mach_aware = transpile(circ, backend=backend, optimization_level=0)

/var/folders/7x/9zsm44vd7h7dhh4b6cdzjjvc0000gn/T/ipykernel_13940/3081846528.py:3: DeprecationWarning: The get_backend() method is deprecated as of qiskit-ibm-runtime 0.24 and will be removed no sooner than 3 months after the release date. Please use backend() instead.
  backend = service.get_backend('ibm_nazca')


In [36]:
# Test circuit transpiled op = 0
circ_trans_op0 = transpile(circ, backend=backend, optimization_level=0)

# Run on IBM device
# backend.run(circ_trans_op0, shots=8192)

# Simulate and calculate fidelities
count_perf = simulate_circuit(circ_mach_aware, reg_sim)
count_error = simulate_circuit(circ_trans_op0, noise_simulator)
print(f'Gates: {circ_trans_op0.count_ops()}')
print(f'Gates: {sum(circ_trans_op0.count_ops().values())}')
print(hellinger_fidelity(count_perf, count_error))

# Find execution fidelity
count_exec = service.job('cvw1t4gbtzcg008n2kb0').result().get_counts()
print(f'Execution Fidelity: {hellinger_fidelity(count_perf, count_exec)}')


Gates: OrderedDict([('rz', 514), ('sx', 221), ('ecr', 69), ('x', 31), ('if_else', 4), ('measure', 3), ('reset', 2)])
Gates: 844
0.9838515393109876
Execution Fidelity: 0.9838850307616219


In [37]:
# Test circuit transpiled op = 1
circ_trans_op1 = transpile(circ, backend=backend, optimization_level=1)

# Run on IBM device
# backend.run(circ_trans_op1, shots=8192)

# Simulate and calculate fidelities
count_perf = simulate_circuit(circ_mach_aware, reg_sim)
count_error = simulate_circuit(circ_trans_op1, noise_simulator)
print(f'Gates: {circ_trans_op1.count_ops()}')
print(f'Gates: {sum(circ_trans_op1.count_ops().values())}')
print(hellinger_fidelity(count_perf, count_error))

# Find execution fidelity
count_exec = service.job('cvw1t787cb40008e8deg').result().get_counts()
print(f'Execution Fidelity: {hellinger_fidelity(count_perf, count_exec)}')

Gates: OrderedDict([('rz', 144), ('sx', 98), ('ecr', 51), ('x', 12), ('if_else', 4), ('measure', 3), ('reset', 2)])
Gates: 314
0.995386605885973
Execution Fidelity: 0.9820611937002418


In [38]:
# Test circuit transpiled op = 2
circ_trans_op2 = transpile(circ, backend=backend, optimization_level=2)

# Run on IBM device
# backend.run(circ_trans_op2, shots=8192)

# Simulate and calculate fidelities
count_perf = simulate_circuit(circ_mach_aware, reg_sim)
count_error = simulate_circuit(circ_trans_op2, noise_simulator)
print(f'Gates: {circ_trans_op2.count_ops()}')
print(f'Gates: {sum(circ_trans_op2.count_ops().values())}')
print(hellinger_fidelity(count_perf, count_error))

# Find execution fidelity
count_exec = service.job('cvw1t919z73000838d9g').result().get_counts()
print(f'Execution Fidelity: {hellinger_fidelity(count_perf, count_exec)}')

Gates: OrderedDict([('rz', 173), ('sx', 108), ('ecr', 43), ('x', 5), ('measure', 3), ('reset', 2)])
Gates: 334
0.9853847609075764
Execution Fidelity: 0.9896098051184447


In [39]:
# Test circuit transpiled op = 3
circ_trans_op3 = transpile(circ, backend=backend, optimization_level=3)

# Run on IBM device
# backend.run(circ_trans_op3, shots=8192)

# Simulate and calculate fidelities
count_perf = simulate_circuit(circ_mach_aware, reg_sim)
count_error = simulate_circuit(circ_trans_op3, noise_simulator)
print(f'Gates: {circ_trans_op3.count_ops()}')
print(f'Gates: {sum(circ_trans_op3.count_ops().values())}')
print(hellinger_fidelity(count_perf, count_error))

# Find execution fidelity
count_exec = service.job('cvw1tas9z73000838da0').result().get_counts()
print(f'Execution Fidelity: {hellinger_fidelity(count_perf, count_exec)}')

Gates: OrderedDict([('rz', 169), ('sx', 108), ('ecr', 43), ('x', 5), ('measure', 3), ('reset', 2)])
Gates: 330
0.9889216832978017
Execution Fidelity: 0.9882869784268031
